# Automated Trading Algorithms


In [ ]:
# Import libraries
% matplotlib inline

import datetime as dt
import numpy as np

from trading_session import * 
from financial_series.everything import * 
from financial_series.stationary_test import adf
from metallic_blue_lizard.neural_net import adaline

## Open a new 'trading session' and get available trading stocks

In [ ]:
# Create instance of trading_session object

ts = TradingSession(user='sd8892',pw='Mexico0r',alias='Trading Session')

In [ ]:
# Get exchange rate
er = ts.getExchangeRates()
er 

In [ ]:
# Print available stocks
available_stocks = ts.getOptions()
print(available_stocks)

# close session
ts.close()

## Download prices 

In [ ]:

# Create list of Stock instances.
obj = list(map(lambda x: 
               Stock(units='MXN',stock_name=yahoo_dict[x],t0='2016/01/01',tf='2017/02/02'), 
               available_stocks))

# Save information in stocks dictionary 
stocks = {}
total_errors = 0
for i,j in zip(obj,available_stocks):
    print('\n- Downloading data: {}'.format(i.name()),end=' ')
    try:
        i.fill()
    except:
        print('\n\t>> Error: cannot download data: {}\n'.format(i.name()))
        total_errors += 1
        continue
    stocks[j] = i
    print('Done.')

# print total of unavailable stock information 
del obj
print('\nTotal Errors: {}'.format(total_errors))

In [ ]:
stocks['ALSEA'].plot([stocks['MEXCHEM']])


In [ ]:
def temp_func(x):
    x.plot();plt.grid()
    plt.show()
    
temp = [temp_func(k) for k in stocks.values()]

# Download exchange rate 

In [ ]:
USDMXN = Currency(base='USD',units='MXN',t0='2016/01/01',tf='2017/02/02')
USDMXN.fill()

In [ ]:
USDMXN.plot();plt.grid()

## Identify Correlation with Exchange Rate

In [ ]:
def getReturns(x):
    return x.returns.Adj_close.values

def equalReturnsLength(x,y):
    _x = getReturns(x).shape
    _y = getReturns(y).shape
    return _x[0] == _y[0] 

def getCov(x,y):
    return np.cov(np.matrix([getReturns(x),getReturns(y)]))

def getCorr(x,y):
    cov_mat = getCov(x,y)
    cov = cov_mat[0,1]
    return cov / (np.std(getReturns(x))*np.std(getReturns(y)))

def matchObj(x,y):
    import copy
    df = pd.concat([x.prices.Adj_close, y.prices.Adj_close],1).dropna().copy()
    _x,_y = df.iloc[:,0],df.iloc[:,1]
    temp_x = copy.deepcopy(x)
    temp_x.prices = pd.DataFrame({'Adj_close':_x.values})
    temp_x.calc_returns()
    
    temp_y = copy.deepcopy(y)
    temp_y.prices = pd.DataFrame({'Adj_close':_y.values})
    temp_y.calc_returns()
    return temp_x, temp_y
    
    
def scatterPlotReturns(x,y):
    _x,_y = matchObj(x,y)
    x_d = pd.DataFrame({'x1':getReturns(_x)})
    y_d = pd.DataFrame({'y':getReturns(_y)})
    neuron = adaline(x_data = x_d, y_data = y_d) 
    neuron.train()
    neuron.evaluate()

    plt.plot(neuron.x_data, neuron.y_data,'b.',
             neuron.x_data, neuron.y_estimate, 'r-')
    plt.title('Scatter Plot of Returns and Linear Estimation \nCorrelation: {}'.format(getCorr(_x,_y)))
    plt.xlabel('Returns of : {}'.format(x.name()))
    plt.ylabel('Returns of : {}'.format(y.name()))
    plt.grid()
    plt.show()

In [ ]:
positive_corr = []
negative_corr = []
best_positive_corr = -float('inf')
best_negative_corr = float('inf')
evaluated_stocks = 0

for k in stocks:
    
    temp_x, temp_y = matchObj(USDMXN,stocks[k])
    
    temp = getCorr(temp_x,temp_y)
    
    # save positive corr
    if temp > 0.5:
        positive_corr.append(k)
        
    if temp < -0.5:
        negative_corr.append(k)
        
    if temp > best_positive_corr:
        best_positive_corr = temp
        pos = k
        
    if temp < best_negative_corr:
        best_negative_corr = temp
        neg = k



In [ ]:
scatterPlotReturns(USDMXN,stocks[pos])
scatterPlotReturns(USDMXN,stocks[neg])


## Stationary Tests - ADF

In [ ]:
found = []

for k in stocks:
    print('Evalualting: {}'.format(k))
    temp = (lambda x: adf(x.prices.Adj_close,0,1))(stocks[k])
    print('\n')
    if temp == 1:
        found.append(k)


In [ ]:
for k in found:
    stocks[k].plot();plt.grid();plt.show()

## Perform operations 